In [22]:
import numpy as np
from docplex.mp.model import Model

# from docplex.mp.callbacks.cb_mixin import ConstraintCallbackMixin
# from docplex.mp.callbacks.cb_mixin import LazyConstraintCallback
import matplotlib.pyplot as plt

In [23]:
import cplex
from cplex.callbacks import UserCutCallback, LazyConstraintCallback
import multiprocessing
import time
import sys
import os
import itertools
import cplex
import numpy as np

In [24]:
# ユークリッド距離の計算
import numpy as np

# 修正後のユークリッド距離の計算関数
def compute_distances(demand_points, candidate_sites):
    D, J = len(demand_points), len(candidate_sites)  # ここで D, J を定義
    distances = np.zeros((D, J))
    for d in range(D):
        for j in range(J):
            distances[d, j] = np.sqrt(
                (demand_points[d][0] - candidate_sites[j][0]) ** 2
                + (demand_points[d][1] - candidate_sites[j][1]) ** 2
            )
    return distances


In [25]:
def compute_wij_matrix(distances, alpha=0, beta=0.1):
    wij_matrix = np.exp(alpha - beta * distances)
    return wij_matrix

In [26]:
def compute_Ui_L(wij_matrix, J_L):
    """
    リーダーの既存施設による U_i^L を計算する関数

    Parameters:
        wij_matrix (np.array): D × J の w_ij の重み行列
        J_L (set): リーダーが既に持っている施設のインデックス集合

    Returns:
        np.array: 各需要点 i に対する U_i^L のベクトル
    """
    D, _ = wij_matrix.shape  # D: 需要点の数, J: 候補施設の数

    # J_L が空なら影響はゼロ
    if not J_L:
        return np.zeros(D)

    # 各需要点 i に対して、リーダーの施設 j ∈ J_L からの重みを合計する
    utility_vector = np.zeros(D)
    for j in J_L:
        # 列 j は、施設 j が各需要点に与える重み
        utility_vector += wij_matrix[:, j]

    return utility_vector


def compute_Ui_F(wij_matrix, J_F):
    """
    リーダーの既存施設による U_i^L を計算する関数

    Parameters:
        wij_matrix (np.array): D × J の w_ij の重み行列
        J_L (set): リーダーが既に持っている施設のインデックス集合

    Returns:
        np.array: 各需要点 i に対する U_i^L のベクトル
    """
    D, _ = wij_matrix.shape  # D: 需要点の数, J: 候補施設の数

    # J_L が空なら影響はゼロ
    if not J_F:
        return np.zeros(D)

    # 各需要点 i に対して、リーダーの施設 j ∈ J_L からの重みを合計する
    utility_vector = np.zeros(D)
    for j in J_F:
        # 列 j は、施設 j が各需要点に与える重み
        utility_vector += wij_matrix[:, j]

    return utility_vector


In [27]:
def compute_L(h_i, Ui_L, Ui_F, wij, x, y):
    """
    関数 L(x, y) を計算する

    Parameters:
        h (np.array): 需要点ごとの人口密度ベクトル (D,)
        Ui_L (np.array): 各需要点におけるリーダーの影響度 (D,)
        Ui_F (np.array): 各需要点におけるフォロワーの影響度 (D,)
        wij (np.array): 需要点と施設候補の重み行列 (D, J)
        x (np.array): リーダーが選択した施設配置 (J,)
        y (np.array): フォロワーが選択した施設配置 (J,)

    Returns:
        float: L(x, y) の計算結果
    """
    numerator = Ui_L + (wij @ x)  # 分子: リーダーの影響度 + 選択した施設の影響
    denominator = Ui_L + Ui_F + (wij @ np.maximum(x, y))  # 分母: 総合影響度

    return np.sum(h_i * (numerator / denominator))


### 大枠から作っていきますか

30秒たっても解けない場合は強制ストップ

In [28]:
def run_solve_s_cflp():
    """solve_s_cflp() を実行し、出力を 'output.log' に保存"""
    with open("output.log", "w") as f:
        sys.stdout = f  # `print()` の出力をファイルにリダイレクト
        sys.stderr = f  # エラーメッセージも保存
        try:
            solve_s_cflp()  # solve_s_cflp を実行
        except Exception as e:
            print(f"⚠️ solve_s_cflp() failed with error: {e}")
        finally:
            sys.stdout = sys.__stdout__  # 元の標準出力に戻す

def solve_with_timeout(timeout=30):
    """solve_s_cflp() を指定時間内に実行し、超過したら強制終了"""
    # **出力ファイルを事前に作成**
    with open("output.log", "w") as f:
        f.write("=== Starting solve_s_cflp() ===\n")

    process = multiprocessing.Process(target=run_solve_s_cflp)
    process.start()
    process.join(timeout)  # 指定時間（秒）待機

    if process.is_alive():
        print("⚠️ solve_s_cflp() was forcibly stopped due to timeout (30s).")
        process.terminate()  # プロセスを強制終了
        process.join()  # プロセスの後処理
        with open("output.log", "a") as f:
            f.write("\n⚠️ solve_s_cflp() was forcibly stopped due to timeout (30s).\n")


In [ ]:
def solve_s_cflp():
    """S-CFLPのBranch-and-Cutフレームワークを実装"""
    global F, BestSol, theta_LB
    last_cut_key = None
    repeated_cut_count = 0
    
    last_bulcut_key = None
    repeated_bulcut_count = 0
    

    # **初期化**
    initialize_problem()

    # **Branch-and-Cut のループ**
    while F:
        # F から最新の Cplex 問題オブジェクトを取得
        problem = F.pop()
        
        # **連続緩和問題を解く**
        incumbent_solution = solve_relaxation(problem)

        # ✅ **デバッグ: 連続緩和の解を出力**
        print("\n=== 🔍 Debugging RELAXATION solution ===")
        print(f"  θ_hat: {incumbent_solution['theta']:.4f}")
        print(f"  x_hat: {incumbent_solution['x']}")
        print("========================================")

        # **分離問題を解く**
        y_star = separation_problem(incumbent_solution)

        # ✅ **デバッグ: 分離問題の結果を出力**
        print("\n=== 🔍 Debugging SEPARATION Problem ===")
        print(f"  y_star: {y_star}")
        print("=======================================")

        # **カットを追加するかチェック**
        if should_add_cut(incumbent_solution, y_star):
            print("\n==============🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍========================")
            print("\n 🔍 Adding New Submopdular Cuts")
            print("\n==============🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍========================")
            print(f"\nafter should_add_cut, problems in F : {len(F)}")
            new_problem,cut_key = add_cuts(incumbent_solution, y_star, problem)
            
            if cut_key == last_cut_key:
                repeated_cut_count += 1
                print(f"⚠️ [Repeat Detected] submodular Cut key repeated {repeated_cut_count} times.")
            else:
                repeated_cut_count = 0
                last_cut_key = cut_key

            if repeated_cut_count >= 2:
                print("🛑 [STOP] Same submodular cut_key repeated multiple times. No progress.")
                break
            
            # **追加された問題の詳細を出力**
            print_problem_details(new_problem)
            
            
            print("\n==============🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍========================")
            print("\n 🔍 Adding New Bulge Cuts")
            print("\n==============🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍========================")
            
            new_problem,bulcut_key = add_bulge_cut(incumbent_solution, y_star, new_problem)
            
            if bulcut_key == last_bulcut_key:
                repeated_bulcut_count += 1
                print(f"⚠️ [Repeat Detected] bulge Cut key repeated {repeated_bulcut_count} times.")
            else:
                repeated_bulcut_count = 0
                last_bulcut_key = bulcut_key

            if repeated_bulcut_count >= 2:
                print("🛑 [STOP] Same bulge cut_key repeated multiple times. No progress.")
                break
        
            # **追加された問題の詳細を出力**
            print_problem_details(new_problem)
            
            F.append(new_problem)
            print(f"after F.append(new_problem), problems in F : {len(F)}")
        else:
            # **最適整数解なら終了**
            if is_integer_solution(incumbent_solution, theta_LB):
                print(f"\nafter is_integer_solution")
                update_best_solution(incumbent_solution)
                break
            else:
                # **分枝処理を行う**
                problem_0, problem_1 = branch_and_bound(incumbent_solution,problem)
                F.append(problem_0)
                F.append(problem_1)

In [30]:
def print_problem_details(problem):
    """
    CPLEXの問題オブジェクトが何を解こうとしているのかを詳細に出力する関数。

    Parameters:
        problem (Cplex): CPLEXの問題オブジェクト
    """
    print("\n=== 🔍 Debugging New Problem ===")

    # 目的関数の出力
    print("\n📌 目的関数:")
    obj_coeffs = problem.objective.get_linear()
    var_names = problem.variables.get_names()
    obj_func_str = " + ".join(f"{obj_coeffs[i]:.4f} * {var_names[i]}" for i in range(len(var_names)))
    print(f"   min {obj_func_str}")

    # 制約の出力
    print("\n📌 制約:")
    constraint_names = problem.linear_constraints.get_names()
    constraint_senses = problem.linear_constraints.get_senses()
    constraint_rhs = problem.linear_constraints.get_rhs()
    constraints_exprs = problem.linear_constraints.get_rows()

    for i, name in enumerate(constraint_names):
        lhs_expr = " + ".join(f"{constraints_exprs[i].val[j]:.4f} * {var_names[constraints_exprs[i].ind[j]]}"
                              for j in range(len(constraints_exprs[i].ind)))
        sense = {"E": "=", "L": "≤", "G": "≥"}[constraint_senses[i]]
        print(f"   {name}: {lhs_expr} {sense} {constraint_rhs[i]:.4f}")

    # 変数の範囲
    # print("\n📌 変数の範囲:")
    var_lb = problem.variables.get_lower_bounds()
    var_ub = problem.variables.get_upper_bounds()
    for i, name in enumerate(var_names):
        pass
        # print(f"   {name}: {var_lb[i]} ≤ {name} ≤ {var_ub[i]}")

    # print("==============================\n")


In [31]:
def initialize_problem():
    """Branch-and-Cutフレームワークの初期化"""
    global F, BestSol, theta_LB  # グローバル変数として明示

    problem = cplex.Cplex()

    # **最大化問題**
    problem.objective.set_sense(problem.objective.sense.maximize)

    # **変数 x_j の追加**
    x_names = [f"x{j}" for j in range(J)]
    problem.variables.add(names=x_names, types=["C"] * J, lb=[0] * J, ub=[1] * J, obj=[0] * J)

    # **目的変数 θ の追加**
    problem.variables.add(names=["theta"], types=["C"], lb=[0], ub=[1], obj=[1])

    # **施設数制約**
    problem.linear_constraints.add(
        lin_expr=[cplex.SparsePair(ind=x_names, val=[1.0] * J)],
        senses=["E"], rhs=[p], names=["facility_limit"]
    )

    # **F の初期化**
    F = [problem]  # 問題をリストに追加
    BestSol = None  # 最良解リセット
    theta_LB = 0  # 下限値リセット

In [32]:
def solve_relaxation(problem):
    """
    連続緩和問題を解き、インカンベント解 (x_hat, theta_hat) を取得する。

    Parameters:
        F (list): 定式化のセット（Cplex 問題オブジェクトのリスト）

    Returns:
        dict: インカンベント解 {'x': np.array, 'theta': float}
    """

    # 問題を解く
    problem.solve()

    # CPLEX の解の取得（変数名のアンダースコアを削除）
    x_values = np.array(
        problem.solution.get_values([f"x{j}" for j in range(J)])
    )  # "x_0" → "x0"
    theta_value = problem.solution.get_values("theta")

    # インカンベント解を辞書形式で返す
    return {"x": x_values, "theta": theta_value}

In [33]:
def separation_problem(incumbent_solution):
    """
    分離問題を解く。Proposition 5 に基づき、最適なフォロワーの施設選択 ŷ を決定する。

    Parameters:
        incumbent_solution (dict): インカンベント解 {'x': np.array, 'theta': float}

    Returns:
        np.array: 最適なフォロワーの施設配置 y_hat (J,)
    """
    # インカンベント解 (x_hat)
    x_hat = incumbent_solution["x"]

    # 定数 a_i(x_hat) の計算
    a_i = Ui_L + (wij_matrix @ x_hat)

    # 定数 w_i^L(x_hat) および w_i^U(x_hat) の計算
    w_i_L = np.min(Ui_F[:, np.newaxis] + wij_matrix * (1 - x_hat), axis=1)
    w_i_U = np.max(Ui_F[:, np.newaxis] + wij_matrix * (1 - x_hat), axis=1)

    # β(x_hat) の計算
    beta_hat = np.sum(
        h_i[:, np.newaxis]
        * (
            (a_i[:, np.newaxis] * wij_matrix * (1 - x_hat))
            / (
                (a_i[:, np.newaxis] + w_i_U[:, np.newaxis])
                * (a_i[:, np.newaxis] + w_i_L[:, np.newaxis])
            )
        ),
        axis=0,
    )

    # β(x_hat) の降順ソート
    sorted_indices = np.argsort(-beta_hat)  # 降順ソート

    # フォロワーの最適な選択 y_hat
    y_hat = np.zeros(J)
    y_hat[sorted_indices[:r]] = 1  # 上位 r 個を選択

    return y_hat

In [34]:
def should_add_cut_v1(incumbent_solution, y_star):
    """
    カットを追加すべきか判定

    Parameters:
        incumbent_solution (dict): インカンベント解 {'x': np.array, 'theta': float}
        y_star (np.array): フォロワーの最適な施設配置 y_hat (J,)

    Returns:
        bool: カットを追加すべきなら True, そうでなければ False
    """
    # インカンベント解 (x_hat, theta_hat)
    x_hat = incumbent_solution["x"]
    theta_hat = incumbent_solution["theta"]

    # L(x_hat, y_star) を計算
    L_value = compute_L(h_i, Ui_L, Ui_F, wij_matrix, x_hat, y_star)
    
    print("\nChecking should add cut")
    print(f"theta_hat: {theta_hat}, L_value: {L_value}")

    # カット追加判定
    return theta_hat > L_value

## should addについて誤差を緩和してみる

In [ ]:
import numpy as np

def should_add_cut(incumbent_solution, y_star, tol=1e-1):
    """
    カットを追加すべきか判定

    Parameters:
        incumbent_solution (dict): インカンベント解 {'x': np.array, 'theta': float}
        y_star (np.array): フォロワーの最適な施設配置 y_hat (J,)
        tol (float): 誤差許容幅（デフォルトは 0.001）

    Returns:
        bool: カットを追加すべきなら True, そうでなければ False
    """
    # インカンベント解 (x_hat, theta_hat)
    x_hat = incumbent_solution["x"]
    theta_hat = incumbent_solution["theta"]

    # L(x_hat, y_star) を計算
    L_value = compute_L(h_i, Ui_L, Ui_F, wij_matrix, x_hat, y_star)

    print("\nChecking should add cut")
    print(f"theta_hat: {theta_hat}, L_value: {L_value}")
    print(f"差分（theta_hat - L_value）: {theta_hat - L_value:.6f}")

    # カット追加判定：わずかな誤差 (tol) を許容して True を返す
    return theta_hat > L_value


In [36]:
def generate_subsets(J):
    """
    J のすべての部分集合 S ⊆ J を生成する（全探索）

    Parameters:
        J (int): 施設候補の数（candidate_sites の長さ）

    Returns:
        list: すべての部分集合 S のリスト（各要素は set 型）
    """
    candidate_indices = set(range(J))  # 修正: `J` を `int` として受け取る
    subsets = []

    # J のすべての部分集合を生成（空集合から J まで）
    for rr in range(len(candidate_indices) + 1):
        for subset in itertools.combinations(candidate_indices, rr):
            subsets.append(set(subset))  # `set` 型で格納

    return subsets


def compute_L_Y(S, Y):
    """
    L_Y(S) を計算する

    Parameters:
        S (set): リーダーが開設する施設の部分集合
        Y (set): フォロワーが開設する施設の集合

    Returns:
        float: L_Y(S)
    """
    # この関数はあってそう
    # print("=== compute_L_Y ===")
    # print(f"S: {S}")
    # print(f"Y: {Y}")
    # print(f"wij_matrix[:, list(S)] : {wij_matrix[:, list(S)]}")
    # print(f"wij_matrix : {wij_matrix}")
    # print("=====================")

    # wij_matrix[:, list(S)]は D × |S| の行列を返却

    numerator = Ui_L + wij_matrix[:, list(S)].sum(axis=1)
    denominator = Ui_L + Ui_F + wij_matrix[:, list(S.union(Y))].sum(axis=1)
    f_i_Y = numerator / denominator
    return np.sum(h_i * f_i_Y)


def compute_rho_Y(S, k, Y):
    """
    rho_Y(S, k) を計算する

    Parameters:
        S (set): リーダーが開設する施設の部分集合
        k (int): 追加する施設のインデックス
        Y (set): フォロワーの施設配置

    Returns:
        float: rho_Y(S, k)
    """
    return compute_L_Y(S.union({k}), Y) - compute_L_Y(S, Y)

In [37]:
def add_cuts_v1(incumbent_solution, y_star, problem):
    """
    カット（制約）を追加する。Algorithm 1 に基づき、サブモジュラー不等式 (8) を適用し、
    重複しないカットのみ追加する。

    Parameters:
        incumbent_solution (dict): インカンベント解 {'x': np.array, 'theta': float}
        y_star (np.array): フォロワーの最適な施設配置 y_hat (J,)
    """
    global existing_cuts  # 追加されたカットを記録するためのグローバル変数

    x_hat = incumbent_solution["x"]
    theta_hat = incumbent_solution["theta"]

    Y = {j for j in range(J) if y_star[j] == 1}
    J_set = set(range(J))

    # 最も違反する S を探索
    min_cut_value = float("inf")
    best_S = None

    for S in generate_subsets(J):
        S_set = set(S)

        rho_sum_1 = sum(compute_rho_Y(J_set - {k}, k, Y) for k in S_set)
        rho_sum_2 = sum(compute_rho_Y(J_set - {k}, k, Y) * x_hat[k] for k in S_set)
        rho_sum_3 = sum(compute_rho_Y(S_set, k, Y) * x_hat[k] for k in (J_set - S_set))

        cut_value = compute_L_Y(S_set, Y) - rho_sum_1 + rho_sum_2 + rho_sum_3

        if cut_value <= min_cut_value:
            min_cut_value = cut_value
            best_S = S_set

    if best_S is not None:
        x_coeffs_s = {k: compute_rho_Y(J_set - {k}, k, Y) for k in best_S}
        x_coeffs_js = {k: compute_rho_Y(best_S, k, Y) for k in (J_set - best_S)}

        constant_term = compute_L_Y(best_S, Y) - sum(
            compute_rho_Y(J_set - {k}, k, Y) for k in best_S
        )
        rhs_value = -constant_term

        # **カットを一意に識別するためのキーを作成**
        cut_key = (rhs_value, tuple(sorted(x_coeffs_s.items())), tuple(sorted(x_coeffs_js.items())))

        print("   Details of the cut:")
        print(f"   RHS Value      : {rhs_value}")
        print(f"   x_coeffs_s     : {sorted(x_coeffs_s.items())}")
        print(f"   x_coeffs_js    : {sorted(x_coeffs_js.items())}")
        print("==============================\n")
            
        # すでに同じカットが追加されていればスキップ
        if cut_key in existing_cuts:
            print("🚫 [SKIP] Duplicate Cut Detected! Not Adding Again.")
            return problem, cut_key  # 何も追加せずに終了


        # **新しいカットを追加**
        existing_cuts.add(cut_key)  # 記録して重複を防ぐ

        submodular_cut_expr = cplex.SparsePair(
            ind=["theta"] + [f"x{k}" for k in J_set],
            val=[-1.0] + list(x_coeffs_s.values()) + list(x_coeffs_js.values()),
        )

        constraint_str = f"-θ"
        for k, coeff in x_coeffs_s.items():
            constraint_str += f" + ({coeff:.4f})x_{k}"
        for k, coeff in x_coeffs_js.items():
            constraint_str += f" + ({coeff:.4f})x_{k}"
        constraint_str += f" ≥ {rhs_value:.4f}"

        print("\n=== ➕ Added Unique Submodular Cut ===")
        print(f"   {constraint_str}")
        print("==============================\n")

        problem.linear_constraints.add(
            lin_expr=[submodular_cut_expr],
            senses=["G"],
            rhs=[rhs_value],
            names=[f"cut_submodular_{len(problem.linear_constraints.get_names())}"],
        )

    return problem , cut_key # 更新した問題を返す


## 以下4/3に追加。

追加されるカットの係数と変数がおかしい↑

In [ ]:
def add_cuts(incumbent_solution, y_star, problem):
    """
    カット（制約）を追加する。Algorithm 1 に基づき、サブモジュラー不等式 (8) を適用し、
    重複しないカットのみ追加する。

    Parameters:
        incumbent_solution (dict): インカンベント解 {'x': np.array, 'theta': float}
        y_star (np.array): フォロワーの最適な施設配置 y_hat (J,)
    Returns:
        Tuple[problem, cut_key or None]: カットを追加した場合はそのキーも返す
    """
    global existing_cuts  # 追加されたカットを記録するためのグローバル変数

    x_hat = incumbent_solution["x"]
    theta_hat = incumbent_solution["theta"]

    Y = {j for j in range(J) if y_star[j] == 1}
    J_set = set(range(J))

    # 最も違反する S を探索
    min_cut_value = float("inf")
    best_S = None

    for S in generate_subsets(J):
        S_set = set(S)

        rho_sum_1 = sum(compute_rho_Y(J_set - {k}, k, Y) for k in S_set)
        rho_sum_2 = sum(compute_rho_Y(J_set - {k}, k, Y) * x_hat[k] for k in S_set)
        rho_sum_3 = sum(compute_rho_Y(S_set, k, Y) * x_hat[k] for k in (J_set - S_set))

        cut_value = compute_L_Y(S_set, Y) - rho_sum_1 + rho_sum_2 + rho_sum_3

        if cut_value <= min_cut_value:
            min_cut_value = cut_value
            best_S = S_set
            
    print(f"best_S: {best_S}\n\n")

    if best_S is not None:
        x_coeffs_s = {k: compute_rho_Y(J_set - {k}, k, Y) for k in best_S}
        x_coeffs_js = {k: compute_rho_Y(best_S, k, Y) for k in (J_set - best_S)}

        constant_term = compute_L_Y(best_S, Y) - sum(
            compute_rho_Y(J_set - {k}, k, Y) for k in best_S
        )
        rhs_value = -constant_term

        # 統一係数マップを作成（初期化）
        coeff_map = {k: 0.0 for k in J_set}
        for k, v in x_coeffs_s.items():
            coeff_map[k] += v
        for k, v in x_coeffs_js.items():
            coeff_map[k] += v

        # カットのキー（順序付き）
        cut_key = (
            rhs_value,
            tuple((k, coeff_map[k]) for k in sorted(coeff_map))
        )
        
        print("==============================")
        print("Details of the cut:")
        print(f"RHS Value      : {rhs_value}")
        print(f"x_coeffs       : {sorted(coeff_map.items())}")
        print("==============================\n")

        # すでに追加済みならスキップ
        if cut_key in existing_cuts:
            print("🚫 [SKIP] Duplicate submodular Cut Detected! Not Adding Again.")
            return problem, cut_key

        existing_cuts.add(cut_key)

        # 順序に揃えて係数と変数リストを作成
        sorted_keys = sorted(coeff_map)
        ind = ["theta"] + [f"x{k}" for k in sorted_keys]
        val = [-1.0] + [coeff_map[k] for k in sorted_keys]

        submodular_cut_expr = cplex.SparsePair(ind=ind, val=val)

        # 人間可読の出力も順序に揃える
        constraint_str = "-θ"
        for k in sorted_keys:
            constraint_str += f" + ({coeff_map[k]:.4f})x_{k}"
        constraint_str += f" ≥ {rhs_value:.4f}"

        print("\n=== ➕ Added Unique Submodular Cut ===")
        print(f"   {constraint_str}")
        print("==============================\n")

        problem.linear_constraints.add(
            lin_expr=[submodular_cut_expr],
            senses=["G"],
            rhs=[rhs_value],
            names=[f"cut_submodular_{len(problem.linear_constraints.get_names())}"],
        )

        return problem, cut_key

    return problem, None


# ↑要検討。間違ってる可能性の方が高い

## Bulge cut

In [ ]:
def add_bulge_cut(incumbent_solution, y_star, problem):
    """
    バルジ不等式（bulge inequality）を CPLEX 問題に追加する安全版。
    数値異常、重複、ゼロ除算を避ける。
    
    Parameters:
        incumbent_solution (dict): {'x': np.array, 'theta': float}
        y_star (np.array): 0-1 ベクトル y のインカンベント値
        problem: CPLEX 問題オブジェクト
    
    Returns:
        Tuple[problem, cut_key or None]
    """
    global existing_cuts, h_i, wij_matrix, Ui_L, Ui_F

    x_hat = incumbent_solution["x"]
    theta_hat = incumbent_solution["theta"]

    D = len(h_i)
    N = len(x_hat)
    I = range(D)
    J = range(N)

    def compute_L_hat(x, y):
        total = 0
        for i in I:
            num = Ui_L[i] + sum(
                wij_matrix[i, j] * (-y[j] * x[j]**2 + (1 + y[j]) * x[j]) for j in J
            )
            den = Ui_L[i] + Ui_F[i] + sum(
                wij_matrix[i, j] * ((1 - y[j]) * x[j] + y[j]) for j in J
            )
            if den == 0:
                print(f"⚠️ Warning: Denominator zero at i={i}. Skipping term.")
                continue
            total += h_i[i] * (num / den)
        return total

    def compute_gj(j, x, y):
        grad = 0
        for i in I:
            P = Ui_L[i] + Ui_F[i] + sum(
                wij_matrix[i, jj] * ((1 - y[jj]) * x[jj] + y[jj]) for jj in J
            )
            Q = Ui_L[i] + sum(
                wij_matrix[i, jj] * (-y[jj] * x[jj]**2 + (1 + y[jj]) * x[jj]) for jj in J
            )
            if P == 0:
                print(f"⚠️ Warning: P=0 at i={i}, j={j}. Skipping term.")
                continue
            term1 = -wij_matrix[i, j] * (1 - y[j]) * Q / (P**2)
            term2 = wij_matrix[i, j] * (-2 * y[j] * x[j] + 1 + y[j]) / P
            grad += h_i[i] * (term1 + term2)
        return grad

    # L̂(x̂, ŷ)
    L_hat_val = compute_L_hat(x_hat, y_star)

    # 安全チェック
    if not np.isfinite(L_hat_val) or abs(L_hat_val) > 1e6:
        print(f"❌ Skipping bulge cut: Invalid L_hat_val = {L_hat_val}")
        return problem, None

    g = {}
    for j in J:
        gj_val = compute_gj(j, x_hat, y_star)
        if not np.isfinite(gj_val) or abs(gj_val) > 1e6:
            print(f"❌ Skipping bulge cut: Invalid gradient g[{j}] = {gj_val}")
            return problem, None
        g[j] = gj_val

    # カットの構築
    coeff_map = {j: g[j] for j in J}
    
    # 不等式の向きが反対向きになることに注意
    rhs_value = -L_hat_val

    cut_key = (
        round(rhs_value, 6),
        tuple((j, round(coeff_map[j], 6)) for j in sorted(J))
    )

    if cut_key in existing_cuts:
        print("🚫 [SKIP] Duplicate Bulge Cut Detected.")
        return problem, cut_key

    existing_cuts.add(cut_key)

    # CPLEX用のカット追加
    ind = ["theta"] + [f"x{j}" for j in sorted(J)]
    val = [-1.0] + [coeff_map[j] for j in sorted(J)]
    bulge_cut_expr = cplex.SparsePair(ind=ind, val=val)

    constraint_str = "-θ"
    for j in sorted(J):
        constraint_str += f" + ({coeff_map[j]:.4f})x_{j}"
    constraint_str += f" ≥ {rhs_value:.4f}"

    print("\n=== ➕ Added Safe Bulge Cut ===")
    print(f"   {constraint_str}")
    print("==============================\n")

    problem.linear_constraints.add(
        lin_expr=[bulge_cut_expr],
        senses=["G"],
        rhs=[rhs_value],
        names=[f"cut_bulge_{len(problem.linear_constraints.get_names())}"],
    )

    return problem, cut_key


とりあえず次に進む

In [2]:
def is_integer_solution(incumbent_solution, theta_LB, tol=1e-5):
    """
    インカンベント解のリーダー施設配置 x が整数 (0 or 1) であり、
    かつ目的関数値 theta が θ_LB を超えているか判定。

    Parameters:
        incumbent_solution (dict): インカンベント解 {'x': np.array, 'theta': float}
        theta_LB (float): 現在の目的関数の下界
        tol (float): 数値誤差の許容範囲（デフォルト 1e-5）

    Returns:
        bool: すべての x_j が整数であり、かつ θ > θ_LB なら True, そうでなければ False
    """
    # 条件1: 目的関数値 θ が θ_LB を超えているか
    if incumbent_solution["theta"] <= theta_LB:
        return False

    # 条件2: x のすべての要素が 0 or 1 の整数か
    x_values = incumbent_solution["x"]
    if np.all(np.abs(x_values - np.round(x_values)) < tol):  # すべて整数に近い
        return True
    else:
        return False

In [3]:
def update_best_solution(incumbent_solution):
    """
    インカンベント解 (x, theta) を用いて最良解 BestSol を更新し、
    目的関数の下界 θ_LB を更新する。

    Parameters:
        incumbent_solution (dict): インカンベント解 {'x': np.array, 'theta': float}
    """
    global BestSol, θ_LB  # グローバル変数を更新

    # BestSol の更新（x のコピーを保存）
    BestSol = incumbent_solution["x"].copy()

    # θ_LB の更新
    θ_LB = incumbent_solution["theta"]

    print("\n=== Best Solution Updated ===")
    print(f"BestSol: {BestSol}")
    print(f"θ_LB: {θ_LB:.4f}")
    print("==============================\n")

In [4]:
def branch_and_bound(incumbent_solution, problem):
    """
    分枝限定法を適用し、最も整数から遠い x_j を固定して
    2つの新しい定式化を作成し、返す。

    Parameters:
        incumbent_solution (dict): インカンベント解 {'x': np.array, 'theta': float}
        problem (cplex.Cplex): 現在の CPLEX 問題
    Returns:
        tuple: (x_j = 0 の問題, x_j = 1 の問題)
    """
    x_hat = incumbent_solution["x"]

    # **最も整数から遠い x_j を選択**
    fractional_indices = np.where((x_hat > 1e-5) & (x_hat < 1 - 1e-5))[0]
    if len(fractional_indices) == 0:
        print("⚠️ No fractional x found. Branching is not needed.")
        return None, None

    # **最大誤差のある x_j を選択**
    j_star = fractional_indices[np.argmax(np.abs(x_hat[fractional_indices] - 0.5))]

    print(f"\n=== 🔀 Branching on x_{j_star} ===")
    print(f"   Fractional value: x_{j_star} = {x_hat[j_star]:.4f}")
    print("==============================\n")

    # 問題をコピーして2つに分ける
    # 現在の定式化をコピー
    problem_0 = cplex.Cplex(problem)  # x_j = 0 の場合
    problem_1 = cplex.Cplex(problem)  # x_j = 1 の場合

    # **🔹 制約 x_j = 0 を追加**
    problem_0.linear_constraints.add(
        lin_expr=[cplex.SparsePair(ind=[f"x{j_star}"], val=[1.0])],
        senses=["E"],
        rhs=[0.0],
        names=[f"branch_x_{j_star}_0"],
    )

    # **🔹 制約 x_j = 1 を追加**
    problem_1.linear_constraints.add(
        lin_expr=[cplex.SparsePair(ind=[f"x{j_star}"], val=[1.0])],
        senses=["E"],
        rhs=[1.0],
        names=[f"branch_x_{j_star}_1"],
    )

    return problem_0, problem_1


## データ生成関数

In [ ]:
import random

def generate_instance(num_rows_columns, I, J, seed=42):
    """
    Generate I demand points and J candidate facility sites from a grid of size num_rows_columns x num_rows_columns.
    Returns: (demand_points, candidate_sites)
    """
    if seed is not None:
        random.seed(seed)
    
    # すべての格子点を生成（1始まり）
    all_points = [(x, y) for x in range(1, num_rows_columns + 1)
                         for y in range(1, num_rows_columns + 1)]
    
    # ランダムにシャッフル
    random.shuffle(all_points)
    
    # 十分な点があるか確認
    assert I + J <= len(all_points), "Grid is too small for given I and J."
    
    demand_points = all_points[:I]
    candidate_sites = all_points[I:I+J]
    
    return demand_points, candidate_sites
